### Part 1.10 - Extract_data_50000_movies

#### Import the libraries

In [ ]:
# For cleaning and preparing the dataset
# -> dataframe manipulation
# -> text manipulation
# -> Web Scrapping

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import os

# Module to serialize the content produced from the execution of the code

import pickle

# Module to monitor the progress of a python for loop

from tqdm import tqdm_notebook
# Example of Use: tqdm_notebook(examples, desc="Converting examples to features")

#### Tenth 5000 movies (50,000)

Extract data about:

* field 1: Plot Summary
* field 2: Actors
* field 3: Directors
* field 4: IMDB rating
* field 5: Plot Synopsis
* field 6: Reviews

In [ ]:
"""
Open the movie_content urls for the tenth 5000 movies and create the souplist of those URLs
"""
content_ten = []

with open(os.path.join(os.getcwd(), 'movie_content_url\\data_ten_14012020.pkl'), 'rb') as f:
    
    content_ten = pickle.load(f)

print("Number of URLs: {}".format(len(content_ten)))

content_souplist_ten = []

for i in tqdm_notebook(content_ten):
    
    content_souplist_ten.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(content_souplist_ten)))

In [ ]:
"""
Open the synopsis urls of the tenth 5000 movies and create the souplist of those URLs
"""
synopsis_ten = []

with open(os.path.join(os.getcwd(), 'synopsis_url\\synopsis_ten_14012020.pkl'), 'rb') as f:
    
    synopsis_ten = pickle.load(f)

print("Number of URLs: {}".format(len(synopsis_ten)))

synopsis_souplist_ten = []

for i in tqdm_notebook(synopsis_ten):
    
    synopsis_souplist_ten.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(synopsis_souplist_ten)))

In [ ]:
"""
Open the reviews urls of the tenth 5000 movies and create the souplist of those URLs
"""
review_ten = []

with open(os.path.join(os.getcwd(), 'reviews_url\\review_ten_16012020.pkl'), 'rb') as f:
    
    review_ten = pickle.load(f)

print("Number of URLs: {}".format(len(review_ten)))

review_souplist_ten = []

for i in tqdm_notebook(review_ten):
    
    review_souplist_ten.append(BeautifulSoup(i.text))

print("Number of review tags: {}".format(len(review_souplist_ten)))

In [ ]:
"""
Field 1: Extract plot summary
"""
myfield_plot = []
plot_summary = []
index_to_remove_no_plot = []

[myfield_plot.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_ten)]

[[[plot_summary.append(y.text) for y in x.find_all('div', {'class':'summary_text'})] for x in i] if len(i) !=0 else index_to_remove_no_plot.append(myfield_plot.index(i)) for i in myfield_plot]
      
print("Length of Plot Summary list: {}".format(len(plot_summary)))
print("Length of the list with Movies that don't have plot summary: {}".format(len(index_to_remove_no_plot)))
if len(index_to_remove_no_plot) == 0:
    print("None of the movie miss plot")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_plot))

In [ ]:
"""
Field 2: Extract actors
"""
myfield_cast = []
phase_two = []
phase_three = []
actors_list = []
index_to_remove_no_actors = []

[myfield_cast.append(i.find_all('table', {'class':'cast_list'})) for i in tqdm_notebook(content_souplist_ten)]
    
r_one = re.compile(".*name")

[[phase_two.append(j.find_all('a', {'href':r_one})) for j in i] for i in myfield_cast]

[phase_three.append(phase_two[i][1::2]) for i in range(len(phase_two))]

[actors_list.append(list(map(lambda x: x.text.strip(' ').replace('\n', ''), actors))) for actors in phase_three]            
    
index_to_remove_no_actors = [i for i,x in enumerate(myfield_cast) if not x]
        
print("Length of Actors list: {}".format(len(actors_list)))
print("Length of the list with Movies that don't have actors: {}".format(len(index_to_remove_no_actors)))
if len(index_to_remove_no_actors) == 0:
    print("None of the movie miss actors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_actors))

In [ ]:
"""
Field 3: Extract director name(s)
"""
myfield_director = []
director_name = []
index_to_remove_no_directors = []

[myfield_director.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_ten)]

r_name = re.compile(".*name")

[[director_name.append(j.find_all('a', {'href':r_name})) for j in i] for i in myfield_director]
    
director_names = [item[0].text for item in director_name if len(item)!=0]

index_to_remove_no_directors = [i for i,x in enumerate(myfield_director) if not x]

print("Length of Directors list: {}".format(len(director_names)))
print("Length of the list with Movies that don't have directors: {}".format(len(index_to_remove_no_directors)))
if len(index_to_remove_no_directors) == 0:
    print("None of the movie miss directors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_directors))

In [ ]:
"""
Field 4: Extract imdb movie rating
"""
myfield_rating = []
ratings = []
index_to_remove_no_rating = []

[myfield_rating.append(i.find_all('div', {'class':'ratingValue'})) for i in tqdm_notebook(content_souplist_ten)]

[[[ratings.append(y.text) for y in x.find_all('span', {'itemprop':'ratingValue'})] for x in i] for i in myfield_rating]

index_to_remove_no_rating = [i for i,x in enumerate(myfield_rating) if not x]

print("Length of Ratings list: {}".format(len(ratings)))
print("Length of the list with Movies that are not rated: {}".format(len(index_to_remove_no_rating)))
if len(index_to_remove_no_rating) == 0:
    print("None of the movie miss ratings")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_rating))

In [ ]:
"""
Field 5: Extract plot synopsis

Having extracted the synopsis content for each movie, we noticed that roughly more than 50% percent of the movies have a synopsis text.
Thus, in order not to erase almost half of the first 5,000 we will not further proceed in keeping the synopsis text
"""
synopsis_step_one = []
synopsis_step_two = []
synopsis_step_three = []

[synopsis_step_one.append(i.find_all('ul', {'class':'ipl-zebra-list', 'id':'plot-synopsis-content'})) for i in tqdm_notebook(synopsis_souplist_ten)]

[[synopsis_step_two.append(j.find_all('li', {'class':'ipl-zebra-list__item'})) for j in i] for i in synopsis_step_one]

[[synopsis_step_three.append(j.text.strip(' ').replace('\n', '').replace('\\', '')) for j in i] for i in synopsis_step_two]

index_to_remove_no_synopsis = [i for i,x in enumerate(synopsis_step_one) if not x]

print("Length of Synopsis list: {}".format(len(synopsis_step_three)))
print("Length of the list with Movies that don't have synopsis: {}".format(len(index_to_remove_no_synopsis)))
if len(index_to_remove_no_synopsis) == 0:
    print("None of the movies miss a synopsis")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_synopsis))

In [ ]:
"""
Print the movies with no synopsis text.
Result: We observed that many movies missed a synopsis text. Thus, we decided to not use synopsis for furhter analysis (although will be extracted)
"""
print([i for i,x in enumerate(synopsis_step_one) if not x])
print([i for i,x in enumerate(synopsis_step_two) if not x])
print([i for i,x in enumerate(synopsis_step_three) if not x])

# Many of the movies do not have a synopsis text, this is it won't be used for furhter analysis (althpugh will be extracted)

In [ ]:
"""
Field 6: Extract movie reviews
"""
myfield_review_step_one = []
myfield_review_step_two = []
myfield_review_step_three = []

[myfield_review_step_one.append(i.find_all('div', {'class':'lister-list'})) for i in tqdm_notebook(review_souplist_ten)]

[[myfield_review_step_two.append(j.find_all('div', {'class':'text show-more__control'})) for j in i] for i in myfield_review_step_one]

[myfield_review_step_three.append(list(map(lambda x: x.text, reviews))) for reviews in myfield_review_step_two]

index_to_remove_no_review = [i for i,x in enumerate(myfield_review_step_one) if not x]

print("Length of Reviews list: {}".format(len(myfield_review_step_three)))
print("Length of the list with Movies that don't have reviews: {}".format(len(index_to_remove_no_review)))
if len(index_to_remove_no_review) == 0:
    print("None of the movies miss reviews")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_review))

In [ ]:
"""
Print the movies with no user reviews text.
"""
print([i for i,x in enumerate(myfield_review_step_one) if not x])
print([i for i,x in enumerate(myfield_review_step_two) if not x])
print([i for i,x in enumerate(myfield_review_step_three) if not x])

<b> - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - </b>

#### Dataframe creation based on the movie content, synopsis and reviews extracted

In [ ]:
"""
Import the movie_content (the extracted HTML document of the column imdb_url)
Import the reviews content (the extracted HTML document of the column reviews_url)
"""
with open(os.path.join(os.getcwd(), 'movie_content_url\\data_ten_14012020.pkl'), 'rb') as f:
    
    content_ten = pickle.load(f)

print("Number of URLs: {}".format(len(content_ten)))

with open(os.path.join(os.getcwd(), 'reviews_url\\review_ten_16012020.pkl'), 'rb') as f:
    
    review_ten = pickle.load(f)

print("Number of URLs: {}".format(len(review_ten)))

In [ ]:
"""
Below we import the 5 fields extracted previously
However, we we imported the "rating" file, we noticed that they did not downloaded properly. Actually, the respective list was empty,
thus we decided to download again the data in order no to make any other mistake with the index of the movies.
"""
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_ten\\pre-indexed files\\plot_ten_18012020.pkl'), 'rb') as f:
    
    plot = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_ten\\pre-indexed files\\rating_ten_18012020.pkl'), 'rb') as f:
    
    rating = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_ten\\pre-indexed files\\actors_ten_18012020.pkl'), 'rb') as f:
    
    actors = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_ten\\pre-indexed files\\director_ten_18012020.pkl'), 'rb') as f:
    
    director = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_ten\\pre-indexed files\\synopsis_ten_18012020.pkl'), 'rb') as f:
    
    synopsis = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_ten\\pre-indexed files\\reviews_ten_18012020.pkl'), 'rb') as f:
    
    reviews = pickle.load(f)
    
print(len(plot))
print(len(rating))
print(len(actors))
print(len(director))
print(len(synopsis))
print(len(reviews))

print(rating) #empty

In [ ]:
"""
Running the commands below we can spot the number of indexes that should be removed from the data extracted.
"Add a Plot": Denoted that the movies does have a written plot on its IMDB website.
"It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.": Also denotes a non-written synopsis.

Synopsis indexes are not further considered. There are too many movies with no synopsis text and there is no poin in erasing almost half of the first 5000 movies due to missing synopsis text.
"""
matching_add_plot = [s for s, x in enumerate(plot) if "Add a Plot" in x]

print("Number of movies with 'Add Plot': ", len(matching_add_plot), '\n')

matching_add_synopsis = [s for s, x in enumerate(synopsis) if 'It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.' in x]

print("Number of movies with no synopsis text: ", len(matching_add_synopsis))

In [ ]:
"""
Remove the faulty indexes from the data. As faulty is characterized an index with no data relevant to movie content or user review of a movie or with an error 404 web page.
Indexes collected during the data extraction of the 6 fields and from the above python cell that prints number of movies with no written plot.
"""
index_remove=[18,147,184,224,252,414,466,517,567,574,597,626,661,795,806,814,821,884,904,905,927,940,973,989,1032,1047,1063,1068,1136,1143,1174,1281,1308,1310,1312,1317,1321,1322,1323,1325,1326,1337,1339,1408,1409,1426,1437,1654,1685,1703,1772,1777,1784,1807,1813,1850,1882,1893,1931,1938,1963,1964,2025,2033,2099,2133,2169,2202,2247,2248,2337,2419,2468,2485,2510,2574,2575,2576,2633,2668,2797,2847,2868,2888,2917,2923,2951,2992,3009,3051,3094,3153,3154,3194,3231,3304,3306,3350,3485,3606,3686,3713,3736,3804,3818,3996,4055,4056,4093,4098,4119,4335,4385,4400,4405,4418,4422,4431,4439,4440,4441,4442,4443,4447,4449,4454,4463,4467,4469,4470,4471,4472,4473,4475,4476,4477,4479,4484,4493,4495,4496,4497,4500,4501,4503,4505,4507,4508,4509,4510,4512,4514,4515,4622,4652,4715,4718,4719,4720,4721,4722,4723,4724,4725,4770,4873,4874,4881,4887,4920,4943,4976,4986,4987,585,942,1229,1340,1954,2270,2282,2296,2816,3188,4034,4395,4431,4439,4440,4441,4449,4467,4503,10,110,145,372,505,600,621,672,729,731,774,848,914,932,1085,1107,1111,1120,1132,1185,1246,1256,1340,1368,1438,1620,1643,1741,1784,1867,1881,1890,1954,2069,2071,2094,2178,2250,2251,2252,2253,2270,2277,2279,2282,2314,2319,2327,2400,2419,2488,2536,2608,2611,2612,2617,2658,2676,2685,2751,2766,2768,2799,2816,2846,2944,2947,2952,2961,3063,3064,3067,3099,3160,3191,3243,3269,3275,3282,3288,3334,3337,3339,3365,3378,3413,3414,3449,3452,3460,3496,3551,3606,3616,3617,3619,3627,3650,3696,3753,3764,3765,3774,3776,3781,3784,3786,3790,3797,3812,3824,3827,3830,3833,3836,3855,3856,3877,3990,4018,4032,4035,4085,4103,4130,4141,4166,4176,4212,4225,4238,4259,4287,4290,4300,4339,4349,4391,4418,4419,4422,4424,4431,4486,4487,4573,4574,4577,4630,4631,4648,4649,4655,4677,4856,4971,4976]
index_remove = list(dict.fromkeys(index_remove))
print(len(index_remove))
content_index_ten = [i for j, i in enumerate(content_ten) if j not in index_remove]
review_index_ten = [i for j, i in enumerate(review_ten) if j not in index_remove]
print(len(content_index_ten))
print(len(review_index_ten))

In [ ]:
"""
Run this cell once! For every re-execution of this notebook, please use the already pickled file.
"""
# with open(os.path.join(os.getcwd(), '58,000 movies\\movie_ten\\content_index_ten_20012020.pkl'), 'wb') as f:
#     pickle.dump(content_index_ten, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movie_ten\\review_index_ten_20012020.pkl'), 'wb') as f:
#     pickle.dump(review_index_ten, f)

In [ ]:
"""
Import the pruned list of movie_content & reviews web pages. The indexes that have been removed did not contain one or more -
of the following fields: plot, actors, director(s), imdb_rating, review(s)

It is important to re-run the process of extracting the information in order to assure that we have extracted the correct information and text for each movie
Otherwise, we might observe mismatches in the fields assigned to each movie title.
"""
content_souplist_ten = []

with open(os.path.join(os.getcwd(), '58,000 movies\\movies_ten\\content_index_ten_20012020.pkl'), 'rb') as f:
    
    content_index_ten = pickle.load(f)

print("Number of URLs: {}".format(len(content_index_ten)))

for i in tqdm_notebook(content_index_ten):
    
    content_souplist_ten.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(content_souplist_ten)))

#-----------------------------------------------------------------------------------------------------

with open(os.path.join(os.getcwd(), '58,000 movies\\movies_ten\\review_index_ten_20012020.pkl'), 'rb') as f:
    
    review_ten = pickle.load(f)

print("Number of URLs: {}".format(len(review_ten)))

review_souplist_ten = []

for i in tqdm_notebook(review_ten):
    
    review_souplist_ten.append(BeautifulSoup(i.text))

print("Number of review tags: {}".format(len(review_souplist_ten)))

In [ ]:
"""
Field 1: Extract plot summary
"""
myfield_plot = []
plot_summary = []
index_to_remove_no_plot = []

[myfield_plot.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_ten)]

[[[plot_summary.append(y.text) for y in x.find_all('div', {'class':'summary_text'})] for x in i] if len(i) !=0 else index_to_remove_no_plot.append(myfield_plot.index(i)) for i in myfield_plot]
      
print("Length of Plot Summary list: {}".format(len(plot_summary)))
print("Length of the list with Movies that don't have plot summary: {}".format(len(index_to_remove_no_plot)))
if len(index_to_remove_no_plot) == 0:
    print("None of the movie miss plot")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_plot))

In [ ]:
"""
Field 2: Extract actors
"""
myfield_cast = []
phase_two = []
phase_three = []
actors_list = []
index_to_remove_no_actors = []

[myfield_cast.append(i.find_all('table', {'class':'cast_list'})) for i in tqdm_notebook(content_souplist_ten)]
    
r_one = re.compile(".*name")

[[phase_two.append(j.find_all('a', {'href':r_one})) for j in i] for i in myfield_cast]

[phase_three.append(phase_two[i][1::2]) for i in range(len(phase_two))]

[actors_list.append(list(map(lambda x: x.text.strip(' ').replace('\n', ''), actors))) for actors in phase_three]            
    
index_to_remove_no_actors = [i for i,x in enumerate(myfield_cast) if not x]
        
print("Length of Actors list: {}".format(len(actors_list)))
print("Length of the list with Movies that don't have actors: {}".format(len(index_to_remove_no_actors)))
if len(index_to_remove_no_actors) == 0:
    print("None of the movie miss actors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_actors))

In [ ]:
"""
Field 3: Extract director name(s)
"""
myfield_director = []
director_name = []
index_to_remove_no_directors = []

[myfield_director.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_ten)]

r_name = re.compile(".*name")

[[director_name.append(j.find_all('a', {'href':r_name})) for j in i] for i in myfield_director]
    
director_names = [item[0].text for item in director_name]

index_to_remove_no_directors = [i for i,x in enumerate(myfield_director) if not x]

print("Length of Directors list: {}".format(len(director_names)))
print("Length of the list with Movies that don't have directors: {}".format(len(index_to_remove_no_directors)))
if len(index_to_remove_no_directors) == 0:
    print("None of the movie miss directors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_directors))

In [ ]:
"""
Field 4: Extract imdb movie rating
"""
myfield_rating = []
ratings = []
index_to_remove_no_rating = []

[myfield_rating.append(i.find_all('div', {'class':'ratingValue'})) for i in tqdm_notebook(content_souplist_ten)]

[[[ratings.append(y.text) for y in x.find_all('span', {'itemprop':'ratingValue'})] for x in i] for i in myfield_rating]

index_to_remove_no_rating = [i for i,x in enumerate(myfield_rating) if not x]

print("Length of Ratings list: {}".format(len(ratings)))
print("Length of the list with Movies that are not rated: {}".format(len(index_to_remove_no_rating)))
if len(index_to_remove_no_rating) == 0:
    print("None of the movie miss ratings")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_rating))

In [ ]:
"""
Field 5: Extract movie user reviews
"""
myfield_review_step_one = []
myfield_review_step_two = []
myfield_review_step_three = []

[myfield_review_step_one.append(i.find_all('div', {'class':'lister-list'})) for i in tqdm_notebook(review_souplist_ten)]

[[myfield_review_step_two.append(j.find_all('div', {'class':'text show-more__control'})) for j in i] for i in myfield_review_step_one]

[myfield_review_step_three.append(list(map(lambda x: x.text, reviews))) for reviews in myfield_review_step_two]

index_to_remove_no_review = [i for i,x in enumerate(myfield_review_step_one) if not x]

print("Length of Reviews list: {}".format(len(myfield_review_step_three)))
print("Length of the list with Movies that don't have reviews: {}".format(len(index_to_remove_no_review)))
if len(index_to_remove_no_review) == 0:
    print("None of the movies miss reviews")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_review))

In [ ]:
print(len([i for i,x in enumerate(myfield_review_step_one) if not x]))
print([i for i,x in enumerate(myfield_review_step_two) if not x])
print([i for i,x in enumerate(myfield_review_step_three) if not x])

In [ ]:
"""
Run this cell once! For every re-execution of this notebook, please use the already pickled file.
"""
# # Pickle the requests file for further use!

# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_ten\\plot_ten_20012020.pkl'), 'wb') as f:
#     pickle.dump(plot_summary, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_ten\\actors_ten_20012020.pkl'), 'wb') as f:
#     pickle.dump(actors_list, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_ten\\director_ten_20012020.pkl'), 'wb') as f:
#     pickle.dump(director_names, f)

# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_ten\\reviews_ten_24012020.pkl'), 'wb') as f:
#     pickle.dump(myfield_review_step_ten, f)

In [ ]:
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_ten\\rating_ten_20012020.pkl'), 'wb') as f:
#     pickle.dump(ratings, f)

In [ ]:
"""
Remove the faulty indexes from the data. As faulty is characterized an index with no data relevant to movie content or user review of a movie or with an error 404 web page.
Indexes collected during the data extraction of the 6 fields and from the above python cell that prints number of movies with no written plot.
"""
index_remove=[18,147,184,224,252,414,466,517,567,574,597,626,661,795,806,814,821,884,904,905,927,940,973,989,1032,1047,1063,1068,1136,1143,1174,1281,1308,1310,1312,1317,1321,1322,1323,1325,1326,1337,1339,1408,1409,1426,1437,1654,1685,1703,1772,1777,1784,1807,1813,1850,1882,1893,1931,1938,1963,1964,2025,2033,2099,2133,2169,2202,2247,2248,2337,2419,2468,2485,2510,2574,2575,2576,2633,2668,2797,2847,2868,2888,2917,2923,2951,2992,3009,3051,3094,3153,3154,3194,3231,3304,3306,3350,3485,3606,3686,3713,3736,3804,3818,3996,4055,4056,4093,4098,4119,4335,4385,4400,4405,4418,4422,4431,4439,4440,4441,4442,4443,4447,4449,4454,4463,4467,4469,4470,4471,4472,4473,4475,4476,4477,4479,4484,4493,4495,4496,4497,4500,4501,4503,4505,4507,4508,4509,4510,4512,4514,4515,4622,4652,4715,4718,4719,4720,4721,4722,4723,4724,4725,4770,4873,4874,4881,4887,4920,4943,4976,4986,4987,585,942,1229,1340,1954,2270,2282,2296,2816,3188,4034,4395,4431,4439,4440,4441,4449,4467,4503,10,110,145,372,505,600,621,672,729,731,774,848,914,932,1085,1107,1111,1120,1132,1185,1246,1256,1340,1368,1438,1620,1643,1741,1784,1867,1881,1890,1954,2069,2071,2094,2178,2250,2251,2252,2253,2270,2277,2279,2282,2314,2319,2327,2400,2419,2488,2536,2608,2611,2612,2617,2658,2676,2685,2751,2766,2768,2799,2816,2846,2944,2947,2952,2961,3063,3064,3067,3099,3160,3191,3243,3269,3275,3282,3288,3334,3337,3339,3365,3378,3413,3414,3449,3452,3460,3496,3551,3606,3616,3617,3619,3627,3650,3696,3753,3764,3765,3774,3776,3781,3784,3786,3790,3797,3812,3824,3827,3830,3833,3836,3855,3856,3877,3990,4018,4032,4035,4085,4103,4130,4141,4166,4176,4212,4225,4238,4259,4287,4290,4300,4339,4349,4391,4418,4419,4422,4424,4431,4486,4487,4573,4574,4577,4630,4631,4648,4649,4655,4677,4856,4971,4976]
index_remove = list(dict.fromkeys(index_remove))
dataset = pd.read_pickle(os.path.join(os.getcwd(), "dataset_58,000_14012020_latest_version.pkl"))
dataset_ten = dataset.iloc[45000:50000].reset_index(drop=True)
dataset_ten = dataset_ten[~dataset_ten.index.isin(index_remove)]
dataset_ten.shape

In [ ]:
"""
Loading the data extracted.
Those are the most important data source across all the notebook. 
The whole data extraction part was executed to end up with the following 6 files per mini-batch.

The examiner can directly import the files below to review the movie content extracted for the tenth mini-batch.
"""
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_ten\\plot_ten_20012020.pkl'), 'rb') as f:
    
    plot = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_ten\\rating_ten_20012020.pkl'), 'rb') as f:
    
    rating = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_ten\\actors_ten_20012020.pkl'), 'rb') as f:
    
    actors = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_ten\\director_ten_20012020.pkl'), 'rb') as f:
    
    director = pickle.load(f)
        
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_ten\\reviews_ten_24012020.pkl'), 'rb') as f:
    
    reviews = pickle.load(f)
    
assert len(plot) == len(rating) == len(actors) == len(director) == len(reviews)

In [ ]:
dataset_ten['actors'] = actors
dataset_ten['plot'] = plot
dataset_ten['imdb_rating'] = rating
dataset_ten['director'] = director
dataset_ten['reviews'] = reviews
dataset_ten = dataset_ten.drop(["movieId", "imdbId", "synopsis_url"], axis=1)

In [ ]:
index_remove=[]
index_remove=[5,19,21,25,26,27,31,32,56,102,124,126,131,143,144,155,250,253,274,303,324,325,327,332,334,351,352,353,354,361,366,392,417,433,434,441,447,452,453,490,504,506,507,518,521,527,561,571,601,602,615,619,629,632,642,663,689,703,705,709,710,711,726,727,731,732,739,765,767,771,772,788,789,795,822,825,828,838,845,889,909,911,916,932,934,985,990,992,996,1003,1007,1010,1016,1030,1031,1047,1063,1064,1065,1080,1083,1084,1093,1100,1102,1107,1110,1112,1123,1133,1135,1140,1147,1148,1149,1166,1167,1179,1190,1201,1213,1223,1225,1232,1240,1245,1249,1277,1279,1283,1285,1289,1290,1295,1299,1315,1331,1336,1343,1370,1388,1389,1393,1453,1457,1460,1467,1468,1473,1474,1477,1496,1498,1514,1516,1557,1574,1594,1595,1607,1611,1613,1626,1628,1634,1635,1668,1676,1679,1682,1684,1685,1689,1690,1698,1710,1713,1714,1731,1750,1764,1769,1773,1774,1778,1782,1792,1810,1818,1829,1847,1848,1849,1854,1869,1875,1877,1885,1887,1900,1904,1917,1928,1929,1939,1940,1942,1952,1960,1963,1965,1986,1996,2002,2006,2010,2015,2024,2032,2033,2045,2047,2048,2049,2051,2054,2055,2058,2059,2061,2065,2066,2071,2072,2075,2079,2081,2082,2083,2084,2087,2089,2097,2101,2105,2107,2108,2144,2147,2153,2154,2155,2156,2157,2161,2164,2168,2169,2172,2174,2177,2180,2184,2189,2190,2191,2193,2196,2198,2199,2235,2265,2282,2284,2290,2291,2298,2321,2326,2327,2330,2345,2357,2358,2361,2363,2374,2377,2382,2389,2403,2407,2428,2434,2435,2474,2479,2491,2501,2525,2539,2562,2572,2580,2601,2603,2607,2616,2661,2670,2697,2707,2708,2718,2737,2765,2767,2769,2778,2781,2783,2793,2802,2812,2828,2834,2838,2866,2896,2913,2920,2926,2932,2934,2941,2944,2946,2950,2963,2964,2966,2970,2974,2976,2977,2991,3003,3009,3017,3022,3024,3025,3026,3028,3031,3044,3053,3054,3064,3073,3074,3089,3095,3110,3116,3141,3147,3153,3156,3162,3182,3188,3194,3200,3202,3204,3206,3210,3223,3233,3235,3243,3256,3262,3264,3284,3285,3286,3287,3294,3295,3302,3349,3376,3414,3416,3422,3424,3456,3460,3463,3465,3467,3468,3476,3478,3484,3487,3493,3498,3499,3501,3513,3517,3519,3523,3529,3539,3540,3548,3557,3564,3579,3580,3583,3585,3601,3602,3612,3616,3619,3625,3635,3636,3670,3759,3764,3800,3801,3805,3820,3824,3835,3838,3852,3858,3861,3878,3882,3893,3894,3945,3963,3965,3966,3968,3969,3973,3974,3975,3977,3978,3979,3981,3982,3986,3996,4002,4013,4026,4039,4040,4042,4043,4048,4051,4065,4066,4079,4083,4084,4096,4117,4163,4217,4235,4237,4258,4266,4272,4275,4277,4289,4290,4291,4299,4306,4319,4336,4337,4338,4339,4342,4343,4345,4367,4368,4392,4394,4400,4410,4414,4425,4444,4447,4455,4457,4463,4468,4470,4472,4475,4480,4485,4505,4506,4510,4519,4539,4541,4553,4570,4578,4602,4607,4613,4622,4624,4626,4629,4645,4649,4650,4653,4655,4656,4658,4660,4661,4663,4666,4667]
dataset_ten = dataset_ten[~dataset_ten.index.isin(index_remove)]
dataset_ten = dataset_ten[dataset_ten.astype(str)['reviews'] != '[]']
dataset_ten.shape

In [ ]:
"""
This is the table to use in Part 1, and the table that will be concatenated with the rest of the mini-batch tables.
"""
# dataset_ten.to_pickle(os.path.join(os.getcwd(), "58,000 movies\\movie_ten\\dataset_ten_final_25012020.pkl"))